In [ ]:
pip install msal azure-identity azure-keyvault-secrets azure-core azure-ai-translation-text==1.0.0b1

In [ ]:
import msal
from azure.core.credentials import AccessToken
from azure.core.credentials import TokenCredential
from azure.keyvault.secrets import SecretClient
from azure.ai.translation.text import TextTranslationClient, TranslatorCredential
from azure.ai.translation.text.models import InputTextItem
from azure.core.exceptions import HttpResponseError
import datetime

# Constants
CLIENT_ID = "fbcdfca9-b26a-44b1-8e57-e87c1335738a"
TENANT_ID = "b9d9d39f-fdde-4609-804a-6487c0daa5e5"
VAULT_URL = "https://WUS3-kv01-ai-policybased.vault.azure.net/"  # Replace with your Key Vault URL
AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"

# Step 1: Acquire the token interactively using MSAL
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)
scopes = ["https://vault.azure.net/.default"]

# This will prompt the user to sign in via browser
result = app.acquire_token_interactive(scopes=scopes)

# Check if token acquisition succeeded
if "access_token" in result:
    access_token = result['access_token']
    print(f"Access Token:\n{access_token}")
else:
    print("Failed to acquire token.")
    exit(1)

# Step 2: Custom TokenCredential implementation to use MSAL token
class MsalTokenCredential(TokenCredential):
    def __init__(self, token):
        self._token = token
    
    def get_token(self, *scopes, **kwargs):
        return AccessToken(self._token, datetime.datetime.utcnow() + datetime.timedelta(hours=1))

# Step 3: Use the token to authenticate with Azure Key Vault
credential = MsalTokenCredential(access_token)
client = SecretClient(vault_url=VAULT_URL, credential=credential)

# Step 4: Query the secret from Azure Key Vault
secret_name = "translator"
retrieved_secret = client.get_secret(secret_name)

# Print the retrieved secret
print(f"Secret '{secret_name}' value: {retrieved_secret.value}")

# Step 5: Translator Method using Azure Language Translation Library
def translate_text():
    """
    Uses Azure AI Language Translate SDK to translate a given text into a specified language.
    :param text: The text to be translated.
    :param target_language: The language code to translate the text into.
    :return: Translated text or error message if the translation fails.
    """
    # Constants for Azure Translator
    TRANSLATOR_KEY = retrieved_secret.value # Replace with your Azure Translator API key
    TRANSLATOR_ENDPOINT = "https://api.cognitive.microsofttranslator.com/"  # Replace with your Azure Translator API endpoint
    TRANSLATOR_REGION = "WESTUS3"  # Replace with your Translator API region
    try:
        # Step 6: Create a TranslatorCredential using the Translator API Key and Region
        credential = TranslatorCredential(TRANSLATOR_KEY, TRANSLATOR_REGION)
        
        # Step 7: Create a TextTranslationClient instance
        text_translator = TextTranslationClient(endpoint=TRANSLATOR_ENDPOINT, credential=credential)

        # Prompt user for input text and target languages
        text_to_translate = input("Enter the text you want to translate: ")
        target_languages = input("Enter the target language codes (comma-separated, e.g., 'es,it,fr'): ").split(',')

        # Step 8: Set up the text input and translation parameters
        source_language = "en"  # Hardcoded source language for now, could be modified to prompt for it
        input_text_elements = [InputTextItem(text=text_to_translate)]

        # Step 9: Call the translate method to perform translation
        response = text_translator.translate(content=input_text_elements, to=target_languages, from_parameter=source_language)

        # Step 10: Process and print the translations
        translation = response[0] if response else None
        if translation:
            for translated_text in translation.translations:
                print(f"Text was translated to: '{translated_text.to}' and the result is: '{translated_text.text}'.")

    except HttpResponseError as exception:
        print(f"Error Code: {exception.error.code}")
        print(f"Message: {exception.error.message}")

# Step 11: Call the Translator Method
translate_text()